In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nhs-expenditure-salisbury/OCT16.csv
/kaggle/input/nhs-expenditure-salisbury/MAY16.csv
/kaggle/input/nhs-expenditure-salisbury/SEP2017.csv
/kaggle/input/nhs-expenditure-salisbury/OCT2018.csv
/kaggle/input/nhs-expenditure-salisbury/JUL2012.csv
/kaggle/input/nhs-expenditure-salisbury/2015_September.csv
/kaggle/input/nhs-expenditure-salisbury/December2014.csv
/kaggle/input/nhs-expenditure-salisbury/Feb2012.csv
/kaggle/input/nhs-expenditure-salisbury/JUL2018.csv
/kaggle/input/nhs-expenditure-salisbury/Dec2015.csv
/kaggle/input/nhs-expenditure-salisbury/01_JAN2011.csv
/kaggle/input/nhs-expenditure-salisbury/MAR2019.csv
/kaggle/input/nhs-expenditure-salisbury/SEP16.csv
/kaggle/input/nhs-expenditure-salisbury/APRIL2018.csv
/kaggle/input/nhs-expenditure-salisbury/FEB2019.csv
/kaggle/input/nhs-expenditure-salisbury/MAY12.csv
/kaggle/input/nhs-expenditure-salisbury/JAN2018.csv
/kaggle/input/nhs-expenditure-salisbury/2015_April.csv
/kaggle/input/nhs-expenditure-salisbury/dec10.csv
/k

# ETL

In [2]:
# This step may take 3-4 minutes

import glob
import chardet

path = r'../input/nhs-expenditure-salisbury' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    
    # Get encoding for each csv
    rawdata = open(filename, 'rb').read()
    result = chardet.detect(rawdata)
    charenc = result['encoding']

    # Read the csv into a dataframe
    df_per_month = pd.read_csv(filename, index_col=None, header=0, encoding=charenc)
    li.append(df_per_month)

df = pd.concat(li, axis=0, ignore_index=True)

In [3]:
df.head()

,Department Family,Entity,Date,Expense Type,Expense area,Supplier,Transaction Number,Amount,VAT registration Number,Amount,VAT Registration Number,Expense,VAT registration number
0,Department of Health,Salisbury NHS Foundation Trust,07-Oct-16,ENERGY - GAS,Corporate - ETS,CORONA ENERGY,11453549,"£5,115.22",NaN,NaN,NaN,NaN,NaN
1,Department of Health,Salisbury NHS Foundation Trust,07-Oct-16,ENERGY - GAS,Corporate - ETS,CORONA ENERGY,11453549,"£44,834.23",NaN,NaN,NaN,NaN,NaN
2,Department of Health,Salisbury NHS Foundation Trust,07-Oct-16,ENERGY - GAS,Corporate - ETS,CORONA ENERGY,11453549,£255.76,NaN,NaN,NaN,NaN,NaN
3,Department of Health,Salisbury NHS Foundation Trust,07-Oct-16,ENERGY - GAS,Corporate - ETS,CORONA ENERGY,11453549,"£8,966.85",NaN,NaN,NaN,NaN,NaN
4,Department of Health,Salisbury NHS Foundation Trust,07-Oct-16,DRUGS,Clinical Support,ROCHE PRODUCTS LTD,1321887892,"£22,497.95",NaN,NaN,NaN,NaN,NaN


In [4]:
df.tail()

,Department Family,Entity,Date,Expense Type,Expense area,Supplier,Transaction Number,Amount,VAT registration Number,Amount,VAT Registration Number,Expense,VAT registration number
814513,Department of Health,Salisbury NHS Foundation Trust,30-Aug-13,MEDICAL EQUIPMENT 10 YEARS,Capital Projects,OLYMPUS KEYMED LTD,CD1900471,NaN,NaN,22950,NaN,NaN,NaN
814514,Department of Health,Salisbury NHS Foundation Trust,30-Aug-13,MEDICAL EQUIPMENT 10 YEARS,Capital Projects,OLYMPUS KEYMED LTD,CD1900471,NaN,NaN,4590,NaN,NaN,NaN
814515,Department of Health,Salisbury NHS Foundation Trust,30-Aug-13,MEDICAL EQUIPMENT 10 YEARS,Capital Projects,OLYMPUS KEYMED LTD,CD1900471,NaN,NaN,-45.9,NaN,NaN,NaN
814516,Department of Health,Salisbury NHS Foundation Trust,30-Aug-13,DRUGS,Clinical Support and Family Services,ROCHE PRODUCTS LTD,1321607824,NaN,NaN,28075.6,NaN,NaN,NaN
814517,Department of Health,Salisbury NHS Foundation Trust,30-Aug-13,DRUGS,Clinical Support and Family Services,ROCHE PRODUCTS LTD,1321607824,NaN,NaN,5615.12,NaN,NaN,NaN


In [5]:
nRow, nCol = df.shape
print('There are {} rows and {} columns in the dataframe.'.format(nRow, nCol))

There are 814518 rows and 13 columns in the dataframe.


## Sorting by Date
First, we need to clean the "Date" column and change the values into sortable values.

In [6]:
# Check for NaN dates

df.Date.isna().sum()

1

In [7]:
df.dropna(axis=0, subset=["Date"], inplace=True)

In [8]:
# Check for empty dates

empty = [index for index, date in enumerate(df.Date.values) if str(date).strip() == '']
df.iloc[empty]

,Department Family,Entity,Date,Expense Type,Expense area,Supplier,Transaction Number,Amount,VAT registration Number,Amount,VAT Registration Number,Expense,VAT registration number


In [9]:
# See how dates are formatted
df.Date.unique()

array(['07-Oct-16', '14-Oct-16', '21-Oct-16', '28-Oct-16', '06-May-16',
       '27-May-16', '13-May-16', '20-May-16', '29-Sep-17', '22-Sep-17',
       '15-Sep-17', '01-Sep-17', '07-Sep-17', '13-Sep-17', '19-Sep-17',
       '05-Oct-18', '19-Oct-18', '26-Oct-18', '12-Oct-18', '06-Jul-12',
       '13-Jul-12', '18-Jul-12', '20-Jul-12', '27-Jul-12', '04-Sep-2015',
       '11-Sep-2015', '18-Sep-2015', '24-Sep-2015', '05-Dec-14',
       '12-Dec-14', '19-Dec-14', '29-Dec-14', '03-Feb-12', '10-Feb-12',
       '24-Feb-12', '17-Feb-12', '27-Jul-18', '20-Jul-18', '13-Jul-18',
       '30-Jul-18', '06-Jul-18', '23-Jul-18', '31-Jul-18', '18-Jul-18',
       '12-Jul-18', '04-Jul-18', '04/12/2015', '18/12/2015', '11/12/2015',
       '29/12/2015', '24/12/2015', '23/12/2015', '28-Jan-11', '21-Jan-11',
       '14-Jan-11', '21/03/2019', '28/03/2019', '15/03/2019',
       '08/03/2019', '22/03/2019', '01/03/2019', '02-Sep-16', '09-Sep-16',
       '16-Sep-16', '23-Sep-16', '30-Sep-16', '27-Apr-18', '20-Apr-18'

In [10]:
# Cast to string

df["Date"] = df["Date"].astype(str)

In [11]:
# Drop values spotted from error messages

df.drop(df.loc[df['Date']=="F"].index, inplace=True)
df.drop(df.loc[df['Date']=="40487"].index, inplace=True)
df.drop(df.loc[df['Date']=="40494"].index, inplace=True)

In [12]:
# Takes around 2 minutes
# Change dates into a sortable format

import dateutil.parser as dparser

df["Date"] = df["Date"].apply(lambda x: dparser.parse(x,fuzzy=True).date())
df.head()

,Department Family,Entity,Date,Expense Type,Expense area,Supplier,Transaction Number,Amount,VAT registration Number,Amount,VAT Registration Number,Expense,VAT registration number
0,Department of Health,Salisbury NHS Foundation Trust,2016-10-07,ENERGY - GAS,Corporate - ETS,CORONA ENERGY,11453549,"£5,115.22",NaN,NaN,NaN,NaN,NaN
1,Department of Health,Salisbury NHS Foundation Trust,2016-10-07,ENERGY - GAS,Corporate - ETS,CORONA ENERGY,11453549,"£44,834.23",NaN,NaN,NaN,NaN,NaN
2,Department of Health,Salisbury NHS Foundation Trust,2016-10-07,ENERGY - GAS,Corporate - ETS,CORONA ENERGY,11453549,£255.76,NaN,NaN,NaN,NaN,NaN
3,Department of Health,Salisbury NHS Foundation Trust,2016-10-07,ENERGY - GAS,Corporate - ETS,CORONA ENERGY,11453549,"£8,966.85",NaN,NaN,NaN,NaN,NaN
4,Department of Health,Salisbury NHS Foundation Trust,2016-10-07,DRUGS,Clinical Support,ROCHE PRODUCTS LTD,1321887892,"£22,497.95",NaN,NaN,NaN,NaN,NaN


In [13]:
df = df.sort_values(by="Date")
df.head()

,Department Family,Entity,Date,Expense Type,Expense area,Supplier,Transaction Number,Amount,VAT registration Number,Amount,VAT Registration Number,Expense,VAT registration number
727009,Department of Health,Salisbury NHS Foundation Trust,2010-04-16,RATES,Facilities Directorate,WILTSHIRE COUNCIL,Apr-10,NaN,NaN,96071.2,NaN,NaN,NaN
727003,Department of Health,Salisbury NHS Foundation Trust,2010-04-16,DRUGS,Clinical Support and Family Services,ROCHE PRODUCTS LTD,1321217701,NaN,NaN,25028.5,NaN,NaN,NaN
727004,Department of Health,Salisbury NHS Foundation Trust,2010-04-16,DRUGS,Clinical Support and Family Services,ROCHE PRODUCTS LTD,1321217701,NaN,NaN,4379.99,NaN,NaN,NaN
727005,Department of Health,Salisbury NHS Foundation Trust,2010-04-16,HEALTHCARE - NHS TRUSTS,Clinical Support and Family Services,ROYAL UNITED HOSPITAL BATH NHS TRUST,10291263,NaN,NaN,71642.5,NaN,NaN,NaN
727006,Department of Health,Salisbury NHS Foundation Trust,2010-04-16,HEALTHCARE - NHS TRUSTS,Medicine,SOUTHAMPTON UNIV HOSPITAL NHS TRUST,7166068,NaN,NaN,26815.2,NaN,NaN,NaN


# Cleaning Other Columns

In [14]:
df.isna().sum()

Department Family               0
Entity                          0
Date                            0
Expense Type                 9707
Expense area                   21
Supplier                        0
Transaction Number              0
 Amount                    748572
VAT registration Number    804799
Amount                      56227
VAT Registration Number    804799
Expense                    795092
VAT registration number    804799
dtype: int64

In [15]:
df.to_csv('expenditure.csv',index=False)